In [4]:
from keras.layers import Conv3D,ConvLSTM2D,Conv3DTranspose
from keras.models import Sequential, Model

In [5]:
class STAE(Model):
  def __init__(self):
    super(STAE, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Input(shape=(227, 227, 10, 1)),
      layers.Conv3D(filters= 128, kernel_size= (11, 11, 1), strides= (4, 4, 1), padding='valid', activation='relu'),
      layers.Conv3D(filters=64, kernel_size=(5, 5, 1), strides=(2, 2, 1), padding='valid', activation='relu')])
    
    self.bottleneck = tf.keras.Sequential([
      layers.ConvLSTM2D(filters=64, kernel_size=(3, 3), strides=1, padding='same', dropout=0.4, recurrent_dropout=0.3, return_sequences=True),
      layers.ConvLSTM2D(filters=32, kernel_size=(3, 3), strides=1, padding='same', dropout=0.3, return_sequences=True),
      layers.ConvLSTM2D(filters=64, kernel_size=(3, 3), strides=1, return_sequences=True, padding='same', dropout=0.5)])
    
    self.decoder = tf.keras.Sequential([
      layers.Conv3DTranspose(filters=128,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='relu'),
      layers.Conv3DTranspose(filters=1,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',activation='relu')])

  def call(self, x):
    encoded = self.encoder(x)
    reduced = self.bottleneck(encoded) 
    decoded = self.decoder(reduced)
    return decoded


In [6]:
def loadModel():
    model = STAE() 
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model